In [ ]:
# Recreate figure analyses and plots - does not require caveCLIENT permissions

In [ ]:
# import packages
import pandas as pd
from matplotlib import pyplot,patches
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import cmocean
import ptitprince as pt


In [ ]:
# import dataframe
# multi_df = pd.read_pickle('pkls/preMN_to_MN_wing_v604.pkl') # preprint era materialization
# properties_df = pd.read_pickle('pkls/wingMN_properties_v604.pkl') # preprint era materialization

multi_df = pd.read_pickle('pkls/preMN_to_MN_wing_011524.pkl')
synapses_df = pd.read_pickle('pkls/synapse_positions_wing_011524.pkl')
properties_df = pd.read_pickle('pkls/mn_properties_wing_011524.pkl')


In [ ]:
# figure 1 plots of size vs synapses

properties_df.head()

In [ ]:
from scipy import stats

indirects_df = properties_df[properties_df.cell_type.str.startswith('D')].reset_index(drop=True)
tension_df = properties_df[properties_df.cell_type.str.startswith(('P','d','t'), na=False)].reset_index(drop=True)
direct_df = properties_df[properties_df.cell_type.str.startswith(('i','iii','hg','b'), na=False)].reset_index(drop=True)

source = direct_df
dx = source.total_area_fixed_um
dy = source.total_inputs

sns.scatterplot(data=source, x=dx, y=dy, s=124)
for point in range(0,source.shape[0]):
     plt.text(dx[point]+300, dy[point], source.cell_type[point], 
              horizontalalignment='left', size='medium', color='black', weight='semibold')

source=source[~source.cell_type.isin(['b1'])]
dx = source.total_area_fixed_um
dy = source.total_inputs

slope, intercept, r, p, std_err = stats.linregress(dx, dy)
def myfunc(dx):
  return slope * dx + intercept
mymodel = list(map(myfunc, dx))
plt.plot(dx, mymodel)

plt.xlim(0,50000)
plt.ylim(0,15000)

# plt.savefig('../wing_xy_indir.svg', format='svg', bbox_inches='tight')
print(r)
plt.show()



In [ ]:
# Extended Data plots
anat_order = ['DLM_1', 'DLM_2', 'DLM_3', 'DLM_4', 'DLM_5', 'DVM_1a', 'DVM_1b', 'DVM_1c', 'DVM_2a', 'DVM_2b', 'DVM_3a', 'DVM_3b', 'PS1', 'PS2', 'dtpmn_u', 'tp2', 'tpn', 'i1', 'i2_u', 'iii1', 'iii3', 'iii4a_u', 'hg1_u', 'hg2',  'hg3', 'hg4', 'b1', 'b2', 'b3_u']


In [ ]:
properties_df.head()

In [ ]:
df1 = properties_df.set_index('cell_type')
df1.reindex(anat_order)

In [ ]:
# number of presynaptic neurons
pre_ids = []
this_syn_df = syn_df[syn_df.pre_pt_root_id.isin(multi_df.index.get_level_values(level='pre_pt_root_id').to_list())]

for mn in properties_df.pt_root_id:
    n = len(this_syn_df[this_syn_df.post_pt_root_id.isin([mn])].pre_pt_root_id.unique())
    pre_ids.append(n)

df1['number_of_neurons'] = pre_ids

In [ ]:
from scipy import stats

source = df1.reindex(anat_order)
dx = source.index #total_area_fixed_um#index
dy = source.number_of_neurons  #total_volume_fixed_um#total_inputs #total_volume_fixed_um
fig = plt.figure(1, figsize = [12,6])

sns.scatterplot(data=source, x=dx, y=dy, s=80)

# source=source[~source.index.isin(['b1'])]
# dx = source.total_area_fixed_um
# dy = source.total_volume_fixed_um

# slope, intercept, r, p, std_err = stats.linregress(dx, dy)
# def myfunc(dx):
#   return slope * dx + intercept
# mymodel = list(map(myfunc, dx))
# plt.plot(dx, mymodel)

# plt.xlim(0,60000)
# plt.ylim(0,15000)
plt.ylim(0,800)

plt.savefig('../wing_numNeurons_11924.svg', format='svg', bbox_inches='tight')
print(r)
plt.show()



In [ ]:
# multi_df is already organized. Cosine similarity metrics are (re-)calculated below

In [ ]:
# plotting code for wing premotor connectome
source = multi_df[anat_order]

fig = plt.figure(1, figsize = [7,12])
sns.set(font_scale= 1)
cmap = cmocean.cm.dense

log_values = np.log10(source +1)
pmn_ids = source.index.get_level_values(level='classification_system').to_list()
mn_ids = source.columns.to_list()
ax = sns.heatmap(log_values, vmin = 0, vmax = 3, xticklabels=mn_ids, yticklabels=pmn_ids, cmap = cmap)

ax.xaxis.set_ticks_position('bottom')
cbar = ax.collections[0].colorbar
plt.title('wing premotor connectivity', size = 20)
cbar.set_label(label = 'log 10 # of synapses', size=16)
plt.xlabel('Wing MNs', size = 16)
plt.ylabel('preMNs', size = 16)
# plt.savefig('../wing_connmat011524.eps', format='eps', bbox_inches='tight')

plt.show()

In [ ]:
source.index.get_level_values(level='classification_system').value_counts()

In [ ]:
# plot proportions from each cell class
df_by_cell_class = multi_df.groupby(level='classification_system').sum().T.rename_axis('MN_label').reset_index()

dict_syn = dict(zip(properties_df.cell_type, properties_df.total_inputs))

df_by_cell_class['synapses_with_fragments'] = df_by_cell_class.MN_label.map(dict_syn)

# dataframe of proportions for plotting
df_by_cell_class_proportions = df_by_cell_class[['local','local_intersegmental','ascending','sensory','descending']].divide(df_by_cell_class.synapses_with_fragments, axis=0)


In [ ]:
df_by_cell_class_proportions.index = df_by_cell_class['MN_label']
df_by_cell_class_proportions['fragment'] = df_by_cell_class_proportions.apply(
    lambda x: 1 - (x['ascending'] + x['descending'] + x['sensory'] + x['local_intersegmental'] + x['local']), axis=1)


In [ ]:
source = df_by_cell_class_proportions

colors_cell_class = ["#916e99","#cea2fd","#069af3","#90e4c1","#95d0fc","#929591", "#650021"]
cmap_cell_class = sns.set_palette(sns.color_palette(colors_cell_class))

#create stacked bar chart
p = source.plot(kind='bar', stacked=True, width = 1, legend = False, cmap = cmap_cell_class, figsize=(7, 2)) 

p.set_ylabel("Proportion of synapses",fontsize = 16)
p.set(ylim=(0, 1))
p.set_xlabel("wing MNs", fontsize = 16)
plt.box(on = False)
# plt.savefig('../wing_classproportions011524.svg', format='svg', bbox_inches='tight')

plt.show()

In [ ]:
df_by_cell_class = df_by_cell_class[:-1]

In [ ]:
# for all wing MNs combined
df_by_cell_class['fragment'] = df_by_cell_class.apply(
    lambda x: x['synapses_with_fragments'] - (x['ascending'] + x['descending'] + x['sensory'] + x['local_intersegmental'] + x['local']), axis=1)

df_total = df_by_cell_class[['local','local_intersegmental','ascending','sensory','descending','fragment']].sum().divide(df_by_cell_class.synapses_with_fragments.sum(), axis=0) # right order


In [ ]:
df_total

In [ ]:
# plot for all wing MNs combined
source = pd.DataFrame(df_total).T

#create stacked bar chart
p = source.plot(kind='bar', stacked=True, width = 1, legend = False, cmap = cmap_cell_class, figsize=(1, 4)) 

p.set_ylabel("Proportion of synapses",fontsize = 16)
p.set(ylim=(0, 1))
p.set_xlabel("all wing MNs", fontsize = 16)
plt.box(on = False)
# plt.savefig('../wingALLmn_classproportions011524.svg', format='svg', bbox_inches='tight')

plt.show()

In [ ]:
# cosine similarity of MNs based on preMN input

In [ ]:
def plot_dendrogram(model, **kwargs):
    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    dend_dict = dendrogram(linkage_matrix, **kwargs)
    
    # sorted order of indices found through clustering
    clustered_order = dend_dict['ivl']
    
    return clustered_order


In [ ]:
def organize_by_cos_short(map_df):
    adj = pd.crosstab(map_df['pre_pt_root_id'],map_df['post_pt_root_id'])
    sim_mat_temp = cosine_similarity(adj.to_numpy().transpose())
    model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat_temp)
    clustered_order = plot_dendrogram(model)#, truncate_mode="level", p=12) # p truncate mode
    clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
    reordered_df = adj.iloc[:,clustered_order]
    sim_mat = cosine_similarity(reordered_df.to_numpy().transpose())
    
    return clustered_order

In [ ]:
def organize_by_cos_long(map_df):
    adj = pd.crosstab(map_df['pre_pt_root_id'],map_df['post_pt_root_id'])
    sim_mat_temp = cosine_similarity(adj.to_numpy())
    model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat_temp)
    clustered_order = plot_dendrogram(model)#, truncate_mode="level", p=12) # p truncate mode
    clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
    reordered_df = adj.iloc[clustered_order,:]
    sim_mat = cosine_similarity(reordered_df.to_numpy())
    
    
    return reordered_df

In [ ]:
syn_df = synapses_df
syn_df.head()

In [ ]:
syn_df.pre_pt_root_id.value_counts()

In [ ]:
pmns = multi_df.index.get_level_values(level='pre_pt_root_id').to_list()


In [ ]:
set(pmns) - set(syn_df.pre_pt_root_id.to_list())

In [ ]:
multi_df.index.get_level_values(level='pre_pt_root_id').value_counts() # should all be one

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

In [ ]:
adj = pd.crosstab(syn_df.pre_pt_root_id,syn_df.post_pt_root_id)
clustered_order = organize_by_cos_short(syn_df)
adj_ordered = adj.iloc[:,clustered_order]

In [ ]:
clust_order = ['DLM_1', 'DLM_2', 'DLM_3', 'DLM_4', 'DLM_5', 
               'DVM_1a', 'DVM_1b', 'DVM_1c', 'DVM_2a', 'DVM_2b', 'DVM_3a', 'DVM_3b', 
                'PS1',
                'dtpmn_u','PS2', 'tp2',
                'iii3','tpn', 'iii1',  
                'i2_u','i1','b3_u',
                'b1', 'b2',
                'hg1_u','hg3', 'hg4','iii4a_u', 
                'hg2']


In [ ]:
dict_mn = dict(zip(properties_df.pt_root_id, properties_df.cell_type))
adj_ordered.rename(columns=dict_mn, inplace=True)

## toggle for anatomical ordering vs module clustering
adj_ordered = adj_ordered[anat_order]
# adj_ordered = adj_ordered[clust_order]

mn_ids = adj_ordered.columns

In [ ]:
sim_mat = cosine_similarity(adj_ordered.to_numpy().transpose())

fig = plt.figure(1, figsize = [6,5])
cmap = cmocean.cm.gray_r
ax = sns.heatmap(sim_mat, xticklabels = mn_ids, yticklabels = '', cmap = cmap)# xticklabels=mn_ids, cmap = cmap)
cbar = ax.collections[0].colorbar
ax.xaxis.set_ticks_position('bottom')

plt.xlabel('', fontsize =16)
plt.title('all preMNs', fontsize = 18)
# plt.savefig('wing_clustsim1115.svg', format='svg', bbox_inches='tight')

plt.show()

In [ ]:
# plot distributions of module preference according to cell class 
# need to create a dataframe where each row contains a unique preMN ID, its cell class, and the fraction of its output onto a single motor cluster, and total # of MN synapses

In [ ]:
# motor clusters
cluster_dict = {
"DLM_clu" : ['DLM_1','DLM_2','DLM_3','DLM_4','DLM_5'],
"DVM_clu" : ['DVM_1a','DVM_1b','DVM_1c','DVM_2a','DVM_2b','DVM_3a','DVM_3b'],
"tension_clu" : ['PS1','PS2','dtpmn_u','tp2'],
"steerA_clu" : ['tpn','iii1','iii3'],
"steerB_clu" : ['i2_u','i1','b3_u'],
"steerC_clu" : ['b2','b1'],
# "hg2_clu" : ['hg2'],
"steerD_clu" : ['hg1_u','hg3','hg4','iii4a_u','hg2']
}
flat_dict = {val:key for key, lst in cluster_dict.items() for val in lst}

In [ ]:
properties_df['cluster'] = properties_df.cell_type.map(flat_dict)
properties_df = properties_df.rename(columns={'MN_id':'post_pt_root_id'})

In [ ]:
syn_df = syn_df.merge(properties_df.rename(columns={'pt_root_id':'post_pt_root_id'})[['post_pt_root_id','cluster']])

In [ ]:
adj = pd.crosstab(syn_df.pre_pt_root_id,syn_df.cluster)
adj.head()

In [ ]:
pref_df = pd.DataFrame(adj.sum(axis=1).reset_index(name='MN_syn'))
pref_df = pref_df.merge(pd.DataFrame(adj.max(axis=1).reset_index(name='top_syn')))
pref_df['proportion'] = pref_df.top_syn / pref_df.MN_syn
pref_df = pref_df.merge(multi_df.reset_index()[['pre_pt_root_id','classification_system']])

pref_df['log_syn'] = np.log(pref_df['MN_syn'])
pref_df['syn_bin'] = pd.cut(pref_df.log_syn,10) # bin number of MN synapses for plotting by hue

pref_df.head()

In [ ]:
ort = 'v' # vertical or horizontally oriented plot
dx = "classification_system"
dy = "proportion"
class_order = ['local', 'local_intersegmental','descending','sensory','ascending']
plot_df = pref_df

pal = sns.color_palette("Greys", n_colors=12)
pal = pal[2:]

f, ax = plt.subplots(figsize=(7, 5))

ax=pt.half_violinplot( x = dx, y = dy, data = plot_df, bw = .2, cut = 0.,
scale = "count", width = .6, inner = None, orient = ort, order=class_order)

ax=sns.stripplot( x = dx, y = dy, data = plot_df, hue="syn_bin", palette = pal,
size = 3, jitter = 1, zorder = 0, orient = ort, order=class_order) # edgecolor = "white"

ax=sns.boxplot( x = dx, y = dy, data = plot_df, color = "black", width = .15, zorder = 10,\
showcaps = True, boxprops = {'facecolor':'none', "zorder":10},\
showfliers=True, whiskerprops = {'linewidth':2, "zorder":10},\
saturation = 1, orient = ort, order=class_order)
plt.ylim(0,1)
plt.title("module preference score")
plt.savefig('../wing_modpref0124.svg', format='svg', bbox_inches='tight')

plt.show()

In [ ]:
# similarity plots, as in Figure 3F
# create a dataframe where the rows are pairs of within-module MNs and each column is the cosine similarity when preMNs are broken up by cell class 

In [ ]:
# create a labeled version of syn_df, with a column for the MN label of postIDs and preMN class for preIDs
labeled_syn_df = syn_df.merge(properties_df.rename(columns={'pt_root_id':'post_pt_root_id'})[['post_pt_root_id', 'cell_type','cluster']],how='left')
labeled_syn_df = labeled_syn_df.merge(multi_df.reset_index()[['pre_pt_root_id','classification_system']],how='left')
labeled_syn_df.head()

In [ ]:
# quantify similarity for within-module pairwise comparisons

In [ ]:
# cos-sim function: input = adjacency matrix with MNs as columns, output = long dataframe with one column of similarity value, and one column for each MN being compared

def adj_to_long_sim (adj_df):
    cs_df = cosine_similarity(adj_df.to_numpy().transpose())
    column_names = adj_df.columns
    df = pd.DataFrame(cs_df, columns=column_names)
    df['mn_1'] = column_names
    df = df.set_index("mn_1")
    df_long = pd.melt(df.reset_index(), id_vars='mn_1', value_vars=adj_df.columns).rename(columns={'post_pt_root_id':'mn_2'})

    # get rid of duplicates and self-comparisons
    df_long = df_long[df_long['mn_1'] != df_long['mn_2']].reset_index(drop=True)
    mns_combined = []
    i=0
    while i < len(df_long):
        if df_long.mn_1[i] > df_long.mn_2[i]:
            # df_long.mns_combined[i] = str(df_long.mn_1[i]) + 'X' + str(df_long.mn_2[i])
            mns_combined.append(str(df_long.mn_1[i]) + 'X' + str(df_long.mn_2[i]))
        else:
            # df_long.mns_combined[i] = str(df_long.mn_2[i]) + 'X' + str(df_long.mn_1[i])
            mns_combined.append(str(df_long.mn_2[i]) + 'X' + str(df_long.mn_1[i]))

        i=i+1
        
    df_long['mns_combined'] = pd.Series(mns_combined)
    df_long = df_long.drop_duplicates(subset='mns_combined').reset_index(drop=True)

    return df_long



In [ ]:
steering_df = adj_ordered.iloc[:,-13:]
steer_tension_df = adj_ordered.iloc[:,-17:]
indir_df = adj_ordered.iloc[:,:12]

In [ ]:
# all

adj_to_measure = steer_tension_df

df_long_all = adj_to_long_sim(adj_to_measure)
df_long_all['cell_class'] = 'all'

# local
df_long_local = adj_to_long_sim(adj_to_measure[adj_to_measure.index.isin(
    multi_df[multi_df.index.get_level_values(level='classification_system').isin(
        ['local'])].index.get_level_values(level='pre_pt_root_id').to_list())])
df_long_local['cell_class'] = 'local'

# local_intersegmental
df_long_intersegmental = adj_to_long_sim(adj_to_measure[adj_to_measure.index.isin(
    multi_df[multi_df.index.get_level_values(level='classification_system').isin(
        ['local_intersegmental'])].index.get_level_values(level='pre_pt_root_id').to_list())])
df_long_intersegmental['cell_class'] = 'local_intersegmental'

# ascending
df_long_ascending = adj_to_long_sim(adj_to_measure[adj_to_measure.index.isin(
    multi_df[multi_df.index.get_level_values(level='classification_system').isin(
        ['ascending'])].index.get_level_values(level='pre_pt_root_id').to_list())])
df_long_ascending['cell_class'] = 'ascending'

# sensory
df_long_sensory = adj_to_long_sim(adj_to_measure[adj_to_measure.index.isin(
    multi_df[multi_df.index.get_level_values(level='classification_system').isin(
        ['sensory'])].index.get_level_values(level='pre_pt_root_id').to_list())])
df_long_sensory['cell_class'] = 'sensory'

# descending
df_long_descending = adj_to_long_sim(adj_to_measure[adj_to_measure.index.isin(
    multi_df[multi_df.index.get_level_values(level='classification_system').isin(
        ['descending'])].index.get_level_values(level='pre_pt_root_id').to_list())])
df_long_descending['cell_class'] = 'descending'

In [ ]:
df_long = pd.concat([df_long_all,df_long_local,df_long_intersegmental,df_long_ascending,df_long_sensory,df_long_descending])

In [ ]:
len(df_long) # should be 2436
df_long

In [ ]:
# cluster?
df_long['module_1'] = df_long.mn_1.map(flat_dict)
df_long['module_2'] = df_long.mn_2.map(flat_dict)

df_long.loc[df_long['module_1'] == df_long['module_2'], 'match'] = 'within'
df_long.loc[df_long['module_1'] != df_long['module_2'], 'match'] = 'across'

In [ ]:
# add a column to indicate within or across module comparisons

# df_long = df_long.merge(properties_df.rename(columns={'MN_label':'mn_1','cluster':'module_1'})[['mn_1','module_1']])
# df_long = df_long.merge(properties_df.rename(columns={'MN_label':'mn_2','cluster':'module_2'})[['mn_2','module_2']])

# df_long.loc[df_long['module_1'] == df_long['module_2'], 'match'] = 'within'
# df_long.loc[df_long['module_1'] != df_long['module_2'], 'match'] = 'across'

In [ ]:
ort = 'v' # vertical or horizontally oriented plot
dx = "cell_class"
dy = "value"
hu = "match"
class_order = ['all', 'local', 'local_intersegmental','descending','sensory','ascending']
plot_df = df_long[df_long.match.isin(['within'])]

pal = sns.color_palette("tab10")

f, ax = plt.subplots(figsize=(15, 5))

ax=sns.stripplot( x = dx, y = dy, data = plot_df, hue=hu, palette = pal,
size = 3, jitter = 1, zorder = 0, orient = ort, order=class_order)

ax=sns.boxplot( x = dx, y = dy, hue=hu, data = plot_df, color = "black", width = .15, zorder = 10,\
showcaps = True, boxprops = {'facecolor':'none', "zorder":10},\
showfliers=True, whiskerprops = {'linewidth':2, "zorder":10},\
saturation = 1, orient = ort, order=class_order)

# ax = sns.catplot(x=dx, y=dy, data = plot_df, hue = 'mns_combined', kind='point')

plt.ylim(0,1)
plt.title("similarity")
# plt.savefig('../wing_steer_sim0118.svg', format='svg', bbox_inches='tight')

plt.show()

In [ ]:
#### Plots for figure 5: hemilineages

In [ ]:
wing_multi_df = multi_df
leg_multi_df = pd.read_pickle('pkls/leg_local_interseg_typed_012524.pkl')

In [ ]:
wing_multi_df.head()

In [ ]:
leg_multi_df.head()

In [ ]:
# just local & intersegmental IDs

wing_df = wing_multi_df[wing_multi_df.index.get_level_values(level='classification_system').str.startswith('l')].reset_index()
leg_df = leg_multi_df[leg_multi_df.index.get_level_values(level='cell_class').isin(['local','intersegmental'])].reset_index()

In [ ]:
leg_df.iloc[:,-69:]#.sum(axis=1)

In [ ]:
# matching up dfs

leg_df = leg_df.rename(columns={'segID':'pre_pt_root_id','NT':'putative_NT'})

# leg MN_syn_total
leg_df['MN_syn_total'] = leg_df.iloc[:,-69:].sum(axis=1)

#rename wing_df columns by module, not MN
df = wing_df.rename(columns=flat_dict, inplace=False)
first_cols = ['pre_pt_root_id', 'pref_mn', 'pref_module', 'cell_type',
        'classification_system', 'putative_NT','MN_syn_total']
module_order = ['DLM_clu','DVM_clu','tension_clu','steerA_clu','steerB_clu','steerC_clu','steerD_clu']
wing_df = df[first_cols + module_order]

In [ ]:
wing_df['limb'] = 'wing'
leg_df['limb'] = 'leg'

merged_premotor = wing_df.merge(leg_df,how='outer')
len(merged_premotor) # should be 1181 + 964 = 2145

In [ ]:
# clean up cell type labels

merged_premotor = merged_premotor.replace(['01A'], '1A')
merged_premotor = merged_premotor.replace(['01B','01B_primary'], '1B')
merged_premotor = merged_premotor.replace(['03A'], '3A')
merged_premotor = merged_premotor.replace(['04B_ventroposterior_loop','04B_lateral','04B_ventroanterior_loop'], '4B')
merged_premotor = merged_premotor.replace(['06A_primary','06A'], '6A')
merged_premotor = merged_premotor.replace(['07B'], '7B')
merged_premotor = merged_premotor.replace(['08A_ventral_mirrored','08A_ventral_post','08A_ventral_rind','08A_far_ventral_rind','08A_ventral','08A_ventral_core','08A_ventral_post_boutons'], '8A')
merged_premotor = merged_premotor.replace(['09A','09Ac','09Ac_primary'], '9A')
merged_premotor = merged_premotor.replace(['11B_u'], '11B')
merged_premotor = merged_premotor.replace(['14Ac','14Ax','14Aa','14Ab','14Ad','14A_primary'], '14A')
merged_premotor = merged_premotor.replace(['hlx16','16B'], '16')
merged_premotor = merged_premotor.replace(['17A_dorsal_post','17A_dorsal_lateral','17A_central_core_post_mirrored','17A_dorsal_post_mirrored','17A_dorsal','17A_central_core_mirrored','17A_dorsal_post_boutons'], '17A')
merged_premotor = merged_premotor.replace(['19A_dorsal_rind','19A_mid'], '19A')
merged_premotor = merged_premotor.replace(['19B_primary'], '19B')
merged_premotor = merged_premotor.replace(['21A_primary'], '21A')
merged_premotor = merged_premotor.replace(['22A_posterior','22A_anterior','22A_posteriordorsal','22A_primary','22A','20A'], '20_22A')
merged_premotor = merged_premotor.replace(['unk3'], 'uk')


In [ ]:
merged_premotor

In [ ]:
g = merged_premotor.groupby('cell_type')

In [ ]:
g.groups.keys()


In [ ]:
wing_sum = []
leg_sum = []

for hl in g.groups.keys():
    
    this_hl = g.get_group(hl)
    wing_sum.append(this_hl[this_hl.limb.isin(['wing'])].MN_syn_total.sum())
    leg_sum.append(this_hl[this_hl.limb.isin(['leg'])].MN_syn_total.sum())

In [ ]:
d = {'hemilineage': g.groups.keys(), 'wing_syn_sum': wing_sum, 'leg_syn_sum' : leg_sum}

hl_syn_totals = pd.DataFrame(data=d)

In [ ]:
hl_syn_proportions = hl_syn_totals.copy()
hl_syn_proportions['wing_syn_sum'] = hl_syn_totals['wing_syn_sum'] / (hl_syn_totals['wing_syn_sum']+hl_syn_totals['leg_syn_sum'])
hl_syn_proportions['leg_syn_sum'] = hl_syn_totals['leg_syn_sum'] / (hl_syn_totals['wing_syn_sum']+hl_syn_totals['leg_syn_sum'])

In [ ]:
hl_order = ['13A','9A','19A','12B','13B','1B','6B','6A','3B','11B','5B','0A',
            '21A', '14A', '24B','8A','16','2A',
            '20_22A','4B', '3A','1A','23B','18B','7B','17A','11A','19B','8B','12A','10B'
]

# less than 10 neurons: 1B, 24B, 10B, 24B and 10B > 100 synapses

In [ ]:
hl_syn_proportions = hl_syn_proportions[hl_syn_proportions.hemilineage.isin(hl_order)]
hl_syn_proportions.hemilineage = hl_syn_proportions.hemilineage.astype("category")
hl_syn_proportions.hemilineage = hl_syn_proportions.hemilineage.cat.set_categories(hl_order)
hl_syn_proportions = hl_syn_proportions.sort_values(["hemilineage"])

In [ ]:
p = hl_syn_proportions.set_index('hemilineage').plot(kind='bar', stacked=True, width = 1, 
                                                     legend = False, cmap = cmap, figsize=(15, 4)) 

plt.savefig('../lhlimbproportions_0126.svg', format='svg', bbox_inches='tight')


In [ ]:
#### connectivity heatmap

heat_df = merged_premotor.copy().drop(['pref_mn','pref_module','classification_system','putative_NT','cell_class','limb','MN_syn_total','pre_pt_root_id'], axis=1)

In [ ]:
# organize columns by module order

wing_mod_order = ['DLM_clu','DVM_clu','tension_clu','steerA_clu','steerB_clu','steerC_clu','steerD_clu']
leg_mod_order = ['coxa_promotion','coxa_rotator_adductor','coxa_posterior','trochanter_extend','trochanter_flex',
                 'femur_reduct','tibia_extend','tibia_ta_flex_A','tibia_ta_flex_B','tibia_ta_flex_C','ltm_dipalpha',
                 'ltm_nondipa','tarsus_depressor_medial','tarsus_depressor_ventralU']
first_cols = ['cell_type']

heat_df[first_cols + leg_mod_order + wing_mod_order]


In [ ]:
# group columns with same name; same cluster

counts = heat_df.iloc[:,3:].groupby(level=0,axis=1).sum()[leg_mod_order+wing_mod_order]

heat_df=pd.concat([heat_df.iloc[:,0:1],counts], axis=1)

In [ ]:
# collapse all hemilineages together
heat_df = heat_df.set_index('cell_type').groupby(level=0,axis=0).sum().reindex(hl_order)


In [ ]:
fig = plt.figure(1, figsize = [7,11])
sns.set(font_scale= 1)
cmap = cmocean.cm.dense

#pmn_ids = reordered_pmn_labels_df.description
# ax = sns.heatmap(all_reorderedLOG_df, vmin = 0, vmax = 3, xticklabels=labeled_adj.columns, yticklabels=sorted_df['class'].to_list(), cmap = cmap)
ax = sns.heatmap(np.log10(heat_df +1), cmap = cmap)

plt.savefig('../hlheatmap_0126.svg', format='svg', bbox_inches='tight')


In [ ]:
## neurotransmitter plot

# new dataframe with MNs as rows
wing_df.putative_NT = wing_df.putative_NT.fillna('unk')
leg_df.putative_NT = leg_df.putative_NT.fillna('unk')
leg_df.loc[leg_df['putative_NT'].isin(['Ach']), 'putative_NT'] = 'ACh'

wing_nt_df = wing_df.iloc[:,5:-1].set_index('putative_NT').groupby(level=0,axis=0).sum()
leg_nt_df = leg_df.iloc[:,3:-2].set_index('putative_NT').groupby(level=0,axis=0).sum()

In [ ]:
nt_df = pd.concat([leg_nt_df,wing_nt_df],axis=1).T[['ACh','unk','Glu','GABA']]

In [ ]:
# drop mn_syn_total row and transform to proportions
nt_df = nt_df.drop(['MN_syn_total'], axis=0)

nt_proportions = nt_df.copy()
nt_proportions['ACh'] = nt_df['ACh']/nt_df.sum(axis=1)
nt_proportions['unk'] = nt_df['unk']/nt_df.sum(axis=1)
nt_proportions['GABA'] = nt_df['GABA']/nt_df.sum(axis=1)
nt_proportions['Glu'] = nt_df['Glu']/nt_df.sum(axis=1)

In [ ]:
nt_proportions

In [ ]:
# plot
# colors_NT = ["#15b01a","#96f97b", "#d8dcd6", "#fdee73", "#fedf08","#f7879a","#cb416b"]
colors_NT = ["#15b01a", "#d8dcd6", "#fedf08", "#cb416b"] # no uncertain colors

cmap = sns.set_palette(sns.color_palette(colors_NT))

p = nt_proportions.plot(kind='bar', stacked=True, width = 1, legend = False, cmap = cmap, figsize=(15, 4)) 

plt.savefig('../ntmns_0126.svg', format='svg', bbox_inches='tight')


In [ ]:
nt_df